In [7]:
from ultralytics import YOLO
model = YOLO('runs/detect/full/weights/full_dataset_100e.pt', )
bin_model = YOLO('runs/detect/birds_planes/weights/birds_planes_200e.pt')

In [8]:
from typing import Dict
import torch
import os
from pprint import pprint

full_keys_dict = {
    0: 'plane',
    24: 'plane',
    9: 'bird'
}
binary_keys_dict = {
    0: 'plane',
    1: 'bird',
}

classes = {
    "plane": ["plane", "aircraft", "jet"],
    "bird": ["bird"]
}


In [14]:
# Per video
from collections import Counter
from tqdm import tqdm


def get_label(fname):
    for key, values in classes.items():
        if any([value in fname for value in values]):
            return key
    return "None"

import cv2
test_vid = 'datasets/04_Test_Datasets/TOTAL_Airplane_over_1km/Test/admin_ch1_20230524104317_20230524104654.mp4'
def _classify_video(model, video_path, label_dict, frame_conf=1e-4, class_conf=0.7):

    results = model(video_path, stream=True, conf=frame_conf, verbose=False)

    def box_helper(result):
        class_conf = torch.stack((result.boxes.cls, result.boxes.conf), dim=1)
        if class_conf.numel() == 0:
            return -1
        # mask = (lambda x: int(x[0]) in label_dict)(class_conf)
        # filtered_boxes = class_conf[mask]
        def filter_boxes(box):
            return int(box[0]) in label_dict

        filtered_boxes = filter(filter_boxes, class_conf)
        filtered_boxes = list(filtered_boxes)
        # print(f"FILTERED LIST: {filtered_boxes}")
        # filtered_boxes = torch.tensor([x.item() for x in filtered_boxes])
        # print(f"BACK AS TENSOR {filtered_boxes}")

        if len(filtered_boxes) == 0:
            return -1
        ret = max(filtered_boxes, key=lambda x: x[1])
        # print(ret)
        return int(ret[0].item())


    # predictions = [
    #     int(r.boxes.cls[
    #         torch.argmax(r.boxes.conf)
    #     ]) if len(r.boxes.cls) > 0 else -1
    #     for r in results
    # ]

    predictions = [
        box_helper(r)
        for r in results
    ]

    # predictions = [
        
    #     [        
    #     int(r.boxes.cls[
    #         torch.argmax(r.boxes.conf)
    #     ]) if len(r.boxes.cls) > 0 else -1
    #     for conf_cls in [
    #         torch.stack((r.boxes.conf, r.boxes.cls), dim=0)[(lambda x: x[1] in label_dict)()]
    #         ]
    #     ][0]
        
    #     for r in results
    # ]
    
    # print("a")
    # predictions = []
    # for r in results:
        
    #     predictions.append(
    #         r.boxes.cls[
    #             torch.argmax(r.boxes.conf)
    #             ]
    #         if len(r.boxes.conf) > 0 else -1
    #     )


    frame_preds = list(Counter(predictions).items())
    frames = sum([num for _, num in frame_preds])
    ratios = sorted([(val, num/frames) for val, num in frame_preds], reverse=True, key=lambda x: x[1])
    if ratios[0][0] != -1 or (
        ratios[0][0] == 1 and 
        ratios[0][1] > class_conf or 
        len(ratios) == 1
        ):
        return ratios[0][0]
    else:
        return ratios[1][0]

def classify_video(model, video_path, frame_conf, class_conf, label_dict):

    prediction = _classify_video(model, video_path, label_dict, frame_conf, class_conf)

    aligned_prediction = label_dict.get(prediction, "None")
    return aligned_prediction



In [15]:
test_dir = 'datasets/04_Test_Datasets/'
def get_test_files(test_dir):
    subdirs = [os.path.join(test_dir, dir) for dir in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, dir))]
    test_files = []
    for subdir in subdirs:
        label = get_label(subdir.lower())
        subdir = os.path.join(subdir, 'Test')
        test_files.extend([(os.path.join(subdir, file), label) for file in
            os.listdir(subdir)
        ])
    return test_files
test_files = get_test_files(test_dir)

In [16]:
def eval_video(model, test_files, frame_conf, class_conf, label_dict):
    files = [t[0] for t in test_files]
    labels = [t[1] for t in test_files]
    labels_dist = {label: len([l for l in labels if l == label]) / len(labels) for label in set(labels)}
    print("starting eval...")
    predictions = []
    for file in tqdm(files):
        predictions.append(classify_video(model, file, frame_conf=frame_conf, class_conf=class_conf, label_dict=label_dict))

    predictions_dist = { 
        label: {
            "total": len([l for l in predictions if l == label]) / len(labels),
            "True_Pos": len([l for l, gt in zip(predictions, labels) if l == label and gt == l]) / len([l for l in predictions if l == label]),
            "False_Pos": 1 - (len([l for l, gt in zip(predictions, labels) if l == label and gt == l]) / len([l for l in predictions if l == label]))
            }
        for label in set(predictions)
        }

    # for f, l, p in zip(files, labels, predictions):
    #     print(f"{f.split('/')[-1]}: label={l}, prediction={p}")
    correct = sum(l == p for l, p in zip(labels, predictions))
    acc = correct /  len(labels)
    print(f"NUM CORRECT: {correct} / {len(labels)} ({acc:.2f})")
    print(f"LABELS DISTRIBUTION:")
    pprint(labels_dist)
    print(f"PREDICTIONS DISTRIBUTION:")
    pprint(predictions_dist)
    return {"acc": acc, "labels_dist": labels_dist, "predictions_dist": predictions_dist}
    


In [17]:
full_model_res = eval_video(model, test_files[:1], frame_conf=0.01, class_conf=0.6, label_dict=full_keys_dict)

starting eval...


100%|██████████| 1/1 [00:00<00:00,  1.51it/s]

NUM CORRECT: 0 / 1 (0.00)
LABELS DISTRIBUTION:
{'bird': 1.0}
PREDICTIONS DISTRIBUTION:
{'plane': {'False_Pos': 1.0, 'True_Pos': 0.0, 'total': 1.0}}


In [18]:
res_full = {}

for cc in [0.6, 0.7, 0.8, 0.9]:
    print(f"\nCLASS CONF = {cc}")
    res_full[cc] = {}
    for fc in [0.001, 0.0005, 0.0001, 0.00005]:
        print(f" - FRAME CONF = {fc}")
        res_full[cc][fc] = eval_video(model, test_files, frame_conf=fc, class_conf=cc, label_dict=full_keys_dict)


CLASS CONF = 0.6
 - FRAME CONF = 0.001
starting eval...


100%|██████████| 81/81 [03:00<00:00,  2.23s/it]


NUM CORRECT: 41 / 81 (0.51)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Pos': 0.8571428571428572,
          'True_Pos': 0.14285714285714285,
          'total': 0.1728395061728395},
 'bird': {'False_Pos': 0.8666666666666667,
          'True_Pos': 0.13333333333333333,
          'total': 0.18518518518518517},
 'plane': {'False_Pos': 0.28846153846153844,
           'True_Pos': 0.7115384615384616,
           'total': 0.6419753086419753}}
 - FRAME CONF = 0.0005
starting eval...


100%|██████████| 81/81 [03:00<00:00,  2.23s/it]


NUM CORRECT: 40 / 81 (0.49)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Pos': 0.8, 'True_Pos': 0.2, 'total': 0.12345679012345678},
 'bird': {'False_Pos': 0.9, 'True_Pos': 0.1, 'total': 0.24691358024691357},
 'plane': {'False_Pos': 0.2941176470588235,
           'True_Pos': 0.7058823529411765,
           'total': 0.6296296296296297}}
 - FRAME CONF = 0.0001
starting eval...


...

NUM CORRECT: 40 / 81 (0.49)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Pos': 0.8, 'True_Pos': 0.2, 'total': 0.12345679012345678},
 'bird': {'False_Pos': 0.9, 'True_Pos': 0.1, 'total': 0.24691358024691357},
 'plane': {'False_Pos': 0.2941176470588235,
           'True_Pos': 0.7058823529411765,
           'total': 0.6296296296296297}}
 - FRAME CONF = 0.0001
starting eval...


100%|██████████| 81/81 [02:57<00:00,  2.19s/it]


NUM CORRECT: 42 / 81 (0.52)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Pos': 0.6666666666666667,
          'True_Pos': 0.3333333333333333,
          'total': 0.07407407407407407},
 'bird': {'False_Pos': 0.9047619047619048,
          'True_Pos': 0.09523809523809523,
          'total': 0.25925925925925924},
 'plane': {'False_Pos': 0.2962962962962963,
           'True_Pos': 0.7037037037037037,
           'total': 0.6666666666666666}}
 - FRAME CONF = 5e-05
starting eval...


100%|██████████| 81/81 [02:56<00:00,  2.18s/it]

NUM CORRECT: 40 / 81 (0.49)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Pos': 0.6, 'True_Pos': 0.4, 'total': 0.06172839506172839},
 'bird': {'False_Pos': 0.9166666666666666,
          'True_Pos': 0.08333333333333333,
          'total': 0.2962962962962963},
 'plane': {'False_Pos': 0.3076923076923077,
           'True_Pos': 0.6923076923076923,
           'total': 0.6419753086419753}}


In [ ]:
res_bin = {}

for cc in [0.6, 0.7, 0.8, 0.9]:
    print(f"\nCLASS CONF = {cc}")
    res_bin[cc] = {}
    for fc in [0.001, 0.0005, 0.0001, 0.00005]:
        print(f" - FRAME CONF = {fc}")
        res_bin[cc][fc] = eval_video(bin_model, test_files, frame_conf=fc, class_conf=cc, label_dict=binary_keys_dict)


CLASS CONF = 0.6
 - FRAME CONF = 0.001
starting eval...


100%|██████████| 81/81 [03:00<00:00,  2.22s/it]


NUM CORRECT: 21 / 81 (0.26)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Pos': 0.9310344827586207,
          'True_Pos': 0.06896551724137931,
          'total': 0.7160493827160493},
 'plane': {'False_Pos': 0.26086956521739135,
           'True_Pos': 0.7391304347826086,
           'total': 0.2839506172839506}}
 - FRAME CONF = 0.0005
starting eval...


100%|██████████| 81/81 [03:02<00:00,  2.25s/it]


NUM CORRECT: 23 / 81 (0.28)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Pos': 0.9152542372881356,
          'True_Pos': 0.0847457627118644,
          'total': 0.7283950617283951},
 'plane': {'False_Pos': 0.18181818181818177,
           'True_Pos': 0.8181818181818182,
           'total': 0.2716049382716049}}
 - FRAME CONF = 0.0001
starting eval...


...

NUM CORRECT: 23 / 81 (0.28)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Pos': 0.9152542372881356,
          'True_Pos': 0.0847457627118644,
          'total': 0.7283950617283951},
 'plane': {'False_Pos': 0.18181818181818177,
           'True_Pos': 0.8181818181818182,
           'total': 0.2716049382716049}}
 - FRAME CONF = 0.0001
starting eval...


100%|██████████| 81/81 [03:03<00:00,  2.26s/it]


NUM CORRECT: 22 / 81 (0.27)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Pos': 0.9310344827586207,
          'True_Pos': 0.06896551724137931,
          'total': 0.7160493827160493},
 'plane': {'False_Pos': 0.21739130434782605,
           'True_Pos': 0.782608695652174,
           'total': 0.2839506172839506}}
 - FRAME CONF = 5e-05
starting eval...


100%|██████████| 81/81 [03:04<00:00,  2.27s/it]

NUM CORRECT: 22 / 81 (0.27)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Pos': 0.9310344827586207,
          'True_Pos': 0.06896551724137931,
          'total': 0.7160493827160493},
 'plane': {'False_Pos': 0.21739130434782605,
           'True_Pos': 0.782608695652174,
           'total': 0.2839506172839506}}


In [ ]:
def flatten(res):
    res_flat = []
    for class_conf, res_1 in res.items():
        for frame_conf, res_2 in res_1.items():
            res_flat.append((res_2['acc'], class_conf, frame_conf))
    return res_flat

res_full_flat = sorted(flatten(res_full), reverse=True, key=lambda x: x[0])
res_bin_flat = sorted(flatten(res_bin), reverse=True, key=lambda x: x[0])

In [ ]:
print(res_full_flat[0])
print(res_bin_flat[0])
print(res_full[0.6][0.001])

(0.4567901234567901, 0.6, 0.001)
(0.2839506172839506, 0.6, 0.0005)
{'acc': 0.4567901234567901, 'labels_dist': {'None': 0.14814814814814814, 'plane': 0.7530864197530864, 'bird': 0.09876543209876543}, 'predictions_dist': {'plane': {'total': 0.41975308641975306, 'True_Pos': 0.7941176470588235, 'False_Pos': 0.20588235294117652}, 'None': {'total': 0.4444444444444444, 'True_Pos': 0.2222222222222222, 'False_Pos': 0.7777777777777778}, 'bird': {'total': 0.13580246913580246, 'True_Pos': 0.18181818181818182, 'False_Pos': 0.8181818181818181}}}


In [ ]:
eval_video(model, test_files, frame_conf=0.00001, class_conf=1, label_dict=full_keys_dict)

starting eval...


100%|██████████| 81/81 [03:02<00:00,  2.25s/it]

NUM CORRECT: 38 / 81 (0.47)
LABELS DISTRIBUTION:
{'None': 0.14814814814814814,
 'bird': 0.09876543209876543,
 'plane': 0.7530864197530864}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Pos': 0.7352941176470589,
          'True_Pos': 0.2647058823529412,
          'total': 0.41975308641975306},
 'bird': {'False_Pos': 0.875, 'True_Pos': 0.125, 'total': 0.19753086419753085},
 'plane': {'False_Pos': 0.12903225806451613,
           'True_Pos': 0.8709677419354839,
           'total': 0.38271604938271603}}


{'acc': 0.4691358024691358,
 'labels_dist': {'None': 0.14814814814814814,
  'plane': 0.7530864197530864,
  'bird': 0.09876543209876543},
 'predictions_dist': {'plane': {'total': 0.38271604938271603,
   'True_Pos': 0.8709677419354839,
   'False_Pos': 0.12903225806451613},
  'None': {'total': 0.41975308641975306,
   'True_Pos': 0.2647058823529412,
   'False_Pos': 0.7352941176470589},
  'bird': {'total': 0.19753086419753085,
   'True_Pos': 0.125,
   'False_Pos': 0.875}}}

In [ ]:
ex_image = 'datasets/TOT/YOLO/images/train/jet8_00002.jpg'
ex_dir = 'testdata'
binary_test_dir = "datasets/ToT_extracted/YOLO/images/test"
dh_test_dir = "datasets/test_image_with_bbox/test"

# res_01 = eval(model, dh_test_dir, full_keys_dict, thresh=0.1)
res_01 = eval(model, dh_test_dir, full_keys_dict, thresh=0.01)
# res_01 = eval(model, dh_test_dir, full_keys_dict, thresh=0.05)
# res_full = eval(model, binary_test_dir, full_keys_dict)
# res_bin = eval(bin_model, binary_test_dir, binary_keys_dict)


image 1/1989 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/datasets/test_image_with_bbox/test/bird_10_00000000.png: 384x640 (no detections), 7.3ms
image 2/1989 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/datasets/test_image_with_bbox/test/bird_10_00000001.png: 384x640 1 balloon, 5.5ms
image 3/1989 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/datasets/test_image_with_bbox/test/bird_10_00000002.png: 384x640 (no detections), 5.3ms
image 4/1989 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/datasets/test_image_with_bbox/test/bird_10_00000003.png: 384x640 1 jet, 5.0ms
image 5/1989 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/datasets/test_image_with_bbox/test/bird_10_00000004.png: 384x640 1 kite, 4.8ms
image 6/1989 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/datasets/test_image_with_bbox/test/bird_10_00000005.png: 384x640 1 jet, 4.9ms
image 7/1989 /home/coinse/Documents/lm/sprin

In [ ]:
res_01

0.3036701860231272

In [ ]:

res_bin_dh = eval(bin_model, dh_test_dir, binary_keys_dict)



image 1/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00014.jpg: 640x384 1 plane, 5.8ms
image 2/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00022.jpg: 640x384 1 plane, 4.9ms
image 3/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00035.jpg: 640x384 1 plane, 4.9ms
image 4/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00039.jpg: 640x384 1 plane, 4.5ms
image 5/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00042.jpg: 640x384 1 plane, 4.8ms
image 6/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/bird1_00006.jpg: 640x384 (no detections), 4.9ms
image 7/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/bird1_00021.jpg: 640x384 1 bird, 4.7ms
image 8/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/bird1_00023.jpg

In [ ]:
eval(bin_model, ex_dir, binary_keys_dict)


image 1/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00014.jpg: 640x384 1 plane, 8.1ms
image 2/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00022.jpg: 640x384 1 plane, 7.4ms
image 3/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00035.jpg: 640x384 1 plane, 7.7ms
image 4/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00039.jpg: 640x384 1 plane, 6.7ms
image 5/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/aircraft10_00042.jpg: 640x384 1 plane, 6.2ms
image 6/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/bird1_00006.jpg: 640x384 (no detections), 4.9ms
image 7/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/bird1_00021.jpg: 640x384 1 bird, 4.7ms
image 8/15 /home/coinse/Documents/lm/spring2024/CS570-Video-Classification/testdata/bird1_00023.jpg

0.3333333333333333